In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.gold_staging.stg_results
USING DELTA
AS
SELECT
  time,
  res.fk,
  res.result.samp_id,
  res.result.as_id,
  res.result.Type,
  res.result.comp,
  res.result.unit,
  res.result.det_id,
  res.result.result,
  res.result.number_of_averaged_samples
FROM workspace.silver_schema.silver_delta_table
LATERAL VIEW EXPLODE(results) AS res;

MERGE INTO workspace.gold_staging.stg_results AS target
USING (
  SELECT * FROM (
    SELECT
      time,
      res.fk,
      res.result.samp_id,
      res.result.as_id,
      res.result.Type,
      res.result.comp,
      res.result.unit,
      res.result.det_id,
      res.result.result,
      res.result.number_of_averaged_samples,
      ROW_NUMBER() OVER (
        PARTITION BY time, res.fk
        ORDER BY time
      ) AS rn
    FROM workspace.silver_schema.silver_delta_table
    LATERAL VIEW EXPLODE(results) AS res
  ) WHERE rn = 1
) AS source
ON target.fk = source.fk
AND target.time = source.time
WHEN MATCHED THEN
  UPDATE SET
    target.samp_id = source.samp_id,
    target.as_id = source.as_id,
    target.Type = source.Type,
    target.comp = source.comp,
    target.unit = source.unit,
    target.det_id = source.det_id,
    target.result = source.result,
    target.number_of_averaged_samples = source.number_of_averaged_samples
WHEN NOT MATCHED THEN
  INSERT (
    time,
    fk,
    samp_id,
    as_id,
    Type,
    comp,
    unit,
    det_id,
    result,
    number_of_averaged_samples
  )
  VALUES (
    source.time,
    source.fk,
    source.samp_id,
    source.as_id,
    source.Type,
    source.comp,
    source.unit,
    source.det_id,
    source.result,
    source.number_of_averaged_samples
  );
